In [ ]:
# pip install pyspark
# pip install findspark 
# pip show pyspark

In [11]:
#import findspark
#findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("lab").getOrCreate()

In [12]:
print(f'The PySpark {spark.version} version is running...')

The PySpark 3.5.1 version is running...


In [13]:
from pyspark.sql.functions import col,column,expr,lit
from pyspark.sql import Row 
from pyspark.sql.types import StructType,StructField,StringType,LongType

In [ ]:
# To Spark (in Python or R), there is no such thing as a Dataset. Everything is a DataFrame 
spark.range(2)

In [ ]:
spark.range(2).collect()

In [ ]:
df = spark.read \
        .format("json") \
        .load("/Applications/MAMP/htdocs/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

In [ ]:
df

In [ ]:
df.schema

In [ ]:
df.printSchema()

In [ ]:
listOfRowObjects = df.collect()

In [ ]:
type(listOfRowObjects)

In [ ]:
len(listOfRowObjects)

In [ ]:
type(listOfRowObjects[0])

In [ ]:
df = spark.createDataFrame([(1, (2,2))], ["a", "b"])
df.show()
df.printSchema(1)
df.printSchema(2)

In [ ]:
df

In [ ]:
df.schema

In [ ]:
from pyspark.sql.functions import col,column,expr

In [ ]:
df.a

In [ ]:
df['a']

In [ ]:
df.select(df.a).show()

In [ ]:
df.select(df['a']).show()

In [ ]:
df.select(col('a')).show()

In [ ]:
# An expression created via the `expr` function is just a DataFrame column reference
# In the simplest case, expr("someCol") is equivalent to col("someCol")
expr('a')

In [ ]:
# All the below are same! 
print(expr('a - 5'))
print(col('a') - 5)
print(expr('a') -5)

In [ ]:
df.columns

### Records and Rows

In [ ]:
df.first()

### Creating Rows

In [ ]:
from pyspark.sql import Row 
myRow = Row("Hello", None, 1, False)

In [ ]:
myRow

In [ ]:
df.createOrReplaceTempView("dfTable")

In [ ]:
spark.sql("select * from dfTable").show(10)

### Creating DataFrames

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType,LongType
myManualSchema = StructType([
    StructField('first_name', StringType(), True),
    StructField('last_name', StringType(), True),
    StructField('age', LongType(), True)
])

alice = Row("Alice", "Henderson", 25)
bob = Row("Bob", "Sanders", 28)
spark.createDataFrame([alice, bob], myManualSchema).show()

In [ ]:
# pip install sparksql-magic
# Load the extension
%load_ext sparksql_magic

In [ ]:
%%sparksql
select * from dfTable limit 250

### select and selectExpr

In [ ]:
df.select('DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME').show(2, False)

In [ ]:
from pyspark.sql.functions import expr, col, column
df.select(
    expr('DEST_COUNTRY_NAME'),
    col('DEST_COUNTRY_NAME'),
    column('DEST_COUNTRY_NAME')).show(2)

In [ ]:
df.select(col('DEST_COUNTRY_NAME'), 'DEST_COUNTRY_NAME')

In [ ]:
df.select(expr('DEST_COUNTRY_NAME as destination')).show(2)

In [ ]:
df.select(expr('DEST_COUNTRY_NAME as destination').alias('alias_destination')).show(2)

In [ ]:
df.selectExpr('DEST_COUNTRY_NAME as destination', 'DEST_COUNTRY_NAME').show(2)

In [ ]:
df.selectExpr('*', '(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withInCountry').show(10)

In [ ]:
# Aggregation over the entire DataFrame
df.selectExpr('avg(count)','count(distinct(DEST_COUNTRY_NAME))').show(2)

### Converting to Spark Types (Literals)

In [ ]:
from pyspark.sql.functions import lit
df.select('*',lit(1).alias('one')).show(2)

In [ ]:
df.select(expr('*'),lit(1).alias('one')).show(2)

### Adding Columns

In [ ]:
df.withColumn('one', lit(1)).show(2)

In [ ]:
# Another way to add a column!
df.withColumn('withInCountry', expr('DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME')).show(2)

In [ ]:
# Another way to rename a column!
df.withColumn('destination',expr('DEST_COUNTRY_NAME')).show(2)

In [ ]:
df.withColumnRenamed('DEST_COUNTRY_NAME','destination').columns

In [ ]:
dfWithLongColName = df.withColumn('Col Name With Space',expr('DEST_COUNTRY_NAME'))

In [ ]:
dfWithLongColName.show(2)

In [ ]:
dfWithLongColName.selectExpr('`Col Name With Space`', '`Col Name With Space` as `new col`').show(2)

In [ ]:
dfWithLongColName.select(col('Col Name With Space')).show(2)

In [ ]:
# We need to escape expressions that use reserved characters/keywords
dfWithLongColName.select(expr('`Col Name With Space`')).show(2)

In [ ]:
df.printSchema

In [ ]:
df.withColumn('count2', col('count').cast('int')).printSchema

In [ ]:
# df.filter(col('count') < 2).show(2)
# df.filter(expr('count') < 2).show(2)
df.filter(expr('count < 2')).show(2)

In [ ]:
df.where('count < 2').show(2)

In [ ]:
df.where(col('count') < 2).show(2)

In [ ]:
df.where(col('count') < 2)\
    .where("ORIGIN_COUNTRY_NAME != 'Croatia'")\
    .show(2)

In [ ]:
df.where(col('count') < 2).where(col('ORIGIN_COUNTRY_NAME') != 'Croatia').show(2)

In [ ]:
df.select('DEST_COUNTRY_NAME','ORIGIN_COUNTRY_NAME').distinct().count()

In [ ]:
df.select('ORIGIN_COUNTRY_NAME').distinct().count()

### Random Samples

In [ ]:
# sampling without replacement, in which a subset of the observations is selected randomly, and once an observation is selected it cannot be selected again. 
# sampling with replacement, in which a subset of observations are selected randomly, and an observation may be selected more than once.
df.sample(False,.9).count()

### Concatenating and Appending Rows (Union)

In [ ]:
from pyspark.sql import Row

lastRow = Row('New Country', 'New Country', 1)
newDF = spark.createDataFrame([lastRow], df.schema)
df.union(newDF).where('DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME').show(300)

### Sorting Rows

In [ ]:
df.sort('count').show(5)

In [ ]:
df.orderBy('count','DEST_COUNTRY_NAME').show(5)

In [ ]:
df.orderBy(col('count'), col('DEST_COUNTRY_NAME')).show(5)

In [ ]:
df.orderBy(expr('count desc')).show(5)

In [ ]:
df.orderBy(col('count').desc(), col('DEST_COUNTRY_NAME').asc()).show(5)

In [ ]:
# For optimization purposes, it's sometimes advisable to sort within each partition before
# another set of transformations
spark.read \
        .format("json") \
        .load("/Applications/MAMP/htdocs/Spark-The-Definitive-Guide/data/flight-data/json/*-summary.json")\
        .sortWithinPartitions('count')

### Repartition & Coalesce
#### https://mrpowers.medium.com/managing-spark-partitions-with-coalesce-and-repartition-4050c57ad5c4

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df.repartition(5)

In [ ]:
rangeDF=spark.range(10)

In [ ]:
rangeDF.rdd.getNumPartitions()

In [ ]:
rangeDF.write.csv('/Users/deepakagrawal/Desktop/data', 'overwrite')

In [ ]:
increasedPartitionedDF=rangeDF.repartition(6)

In [ ]:
increasedPartitionedDF.write.csv('/Users/deepakagrawal/Desktop/data', 'overwrite')

## Chapter 6

In [14]:

df = spark.read.format('csv') \
          .option('header', 'true') \
          .option('inferSchema','true') \
          .load("/Applications/MAMP/htdocs/Spark-The-Definitive-Guide/data/retail-data/by-day/2010-12-01.csv")

In [15]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [25]:
df.schema.fields

[StructField('InvoiceNo', StringType(), True),
 StructField('StockCode', StringType(), True),
 StructField('Description', StringType(), True),
 StructField('Quantity', IntegerType(), True),
 StructField('InvoiceDate', TimestampType(), True),
 StructField('UnitPrice', DoubleType(), True),
 StructField('CustomerID', DoubleType(), True),
 StructField('Country', StringType(), True)]

In [6]:
df.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [9]:
df.describe()

DataFrame[summary: string, InvoiceNo: string, StockCode: string, Description: string, Quantity: string, UnitPrice: string, CustomerID: string, Country: string]

In [ ]:
df.createOrReplaceTempView('dfTable')

In [ ]:
df.show(10, False)

In [ ]:
df.select(lit(5), lit("five"), lit(5.0)).printSchema()

In [ ]:
df.select(lit(5), lit("five"), lit(5.0)).show(10)

In [ ]:
lit('first_name')

In [ ]:
col('first_name')

In [ ]:
column('first_name')

In [ ]:
expr('first_name')

In [33]:
def spark_to_postgres_type(spark_type):
    if spark_type == 'string':
        return 'TEXT'
    elif spark_type == 'integer':
        return 'INTEGER'
    elif spark_type == 'long':
        return 'BIGINT'
    elif spark_type == 'float':
        return 'REAL'
    elif spark_type == 'double':
        return 'DOUBLE PRECISION'
    elif spark_type == 'boolean':
        return 'BOOLEAN'
    elif spark_type == 'date':
        return 'DATE'
    elif spark_type == 'timestamp':
        return 'TIMESTAMP'
    else:
        return 'TEXT'

table_name = 'my_table'
create_table_sql = f"CREATE TABLE {table_name} ("
for field in df.schema.fields:
    field_name = field.name
    field_type = field.dataType.typeName()
    postgres_field_type = spark_to_postgres_type(field_type)
    create_table_sql += f"{field_name} {postgres_field_type}, "

create_table_sql = create_table_sql[:-2] # Remove the last comma and the space
create_table_sql += ')'
print(create_table_sql)

CREATE TABLE my_table (InvoiceNo TEXT, StockCode TEXT, Description TEXT, Quantity INTEGER, InvoiceDate TIMESTAMP, UnitPrice DOUBLE PRECISION, CustomerID DOUBLE PRECISION, Country TEXT)
